# СИГНАЛ ПЛОХОЙ - ПЛОХО ОБРАБАТЫВАЕТСЯ
# МБ ФИЛЬТРЫ ВЫВЕЗУТ

# Вывезли

In [23]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
from scipy.signal import resample, butter, filtfilt

In [24]:
def funcBPFilter(ppg_signal, order, f1, f2, fs):
    fc = [f1, f2]
    b, a = butter(order, fc, btype='bandpass', analog=False, output='ba', fs=fs)
    signalBP = filtfilt(b, a, ppg_signal)
    return signalBP

In [25]:
age_group = {15:1.0,16:1.0,17:1.0,18:1.0,19:1.0,
             20:2.0,21:2.0,22:2.0,23:2.0,24:2.0,
             25:3.0,26:3.0,27:3.0,28:3.0,29:3.0,
             30:4.0,31:4.0,32:4.0,33:4.0,34:4.0,
             35:5.0,36:5.0,37:5.0,38:5.0,39:5.0,
             40:6.0,41:6.0,42:6.0,43:6.0,44:6.0,
             45:7.0,46:7.0,47:7.0,48:7.0,49:7.0,
             50:8.0,51:8.0,52:8.0,53:8.0,54:8.0,
             55:9.0,56:9.0,57:9.0,58:9.0,59:9.0,
             60:10.0,61:10.0,62:10.0,63:10.0,64:10.0,
             65:11.0,66:11.0,67:11.0,68:11.0,69:11.0,
             70:12.0,71:12.0,72:12.0,73:12.0,74:12.0,
             75:13.0,76:13.0,77:13.0,78:13.0,79:13.0,
             80:14.0,81:14.0,82:14.0,83:14.0,84:14.0,
             85:15.0,86:15.0,87:15.0,88:15.0,89:15.0,
             90:16.0,91:16.0,92:16.0,93:16.0,94:16.0,
             95:17.0,96:17.0,97:17.0,98:17.0,99:17.0,100:17.0,
             }

In [26]:
subjects = pd.read_excel(r"D:\Proga\AML\datasets\lucilacapurro\PPG_Signal_Dataset\subjects_metadata.xlsx",dtype = {"ID":str})
#subjects.dropna(inplace=True)
subjects = subjects[subjects['observations'].isna()]
subjects = subjects.reset_index()
subjects = subjects[["subjectcode","age"]]
subjects["age"] = subjects["age"].map(age_group)

In [27]:
ids = subjects["subjectcode"].values
age_groups = subjects["age"].values

In [28]:
result = []
result_metadata = []
result_signals = {}

target_fs = 1000
five_minute_samples = 300 * target_fs
new_id = 1096

for id in ids:
    new_id += 1 
    try:
        df_ppg = pd.read_csv("D:\\Proga\\AML\\datasets\\lucilacapurro\\PPG_Signal_Dataset\\data\\"+id+".csv")
        signal_ppg = df_ppg['PPG'].values
        #print(record.__dict__)
        #print(record.p_signal.shape)
        #channel_index = record.sig_name.index('NIBP')
        signal = signal_ppg.reshape(-1, 1).astype(np.float32)
        fs = target_fs
        print(len(signal[0]))

        if fs != target_fs:
            num_samples = int(len(signal) * target_fs / fs)
            signal = resample(signal, num_samples).astype(np.float32)
            fs = target_fs

        signal = signal[:five_minute_samples].flatten()

        signal = funcBPFilter(signal, order=2, f1=0.5, f2=4.0, fs=fs)
        
        try:
            wd, m = hp.process(signal, fs)
            idx = int(subjects[subjects["subjectcode"] == id].index.to_list()[0])
            age_group = age_groups[idx]
            
            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                **m
            })
            
            result_signals[new_id] = signal
            
            print((new_id, id, age_group, signal, m))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")

1
(1097, 'S02', 1.0, array([ 1.71532606e+04,  1.71787948e+04,  1.72032496e+04, ...,
       -2.62158893e+00, -2.19116248e+00, -1.81041995e+00]), {'bpm': 121.05535437144336, 'ibi': 495.64102564102564, 'sdnn': 76.22267889831241, 'sdsd': 75.01607559472261, 'rmssd': 103.93533492587517, 'pnn20': 0.7846153846153846, 'pnn50': 0.47692307692307695, 'hr_mad': 46.0, 'sd1': 73.22015598592746, 'sd2': 62.23501364245791, 's': 14315.789752304403, 'sd1/sd2': 1.1765106440978632, 'breathingrate': 0.23333333333333334})
Process end: S02
1
(1098, 'S03', 1.0, array([-579.05417305, -612.77848565, -646.78438372, ...,    6.67544731,
          5.61433689,    4.66688971]), {'bpm': 175.00628614533568, 'ibi': 342.8448275862069, 'sdnn': 25.831105986088293, 'sdsd': 9.96357427445074, 'rmssd': 16.71591951757124, 'pnn20': 0.21965317919075145, 'pnn50': 0.005780346820809248, 'hr_mad': 17.0, 'sd1': 11.818693367453792, 'sd2': 34.589334563636044, 's': 1284.285398398986, 'sd1/sd2': 0.3416860577560474, 'breathingrate': 0.166666

c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\heartpy\analysis.py:677: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  interp = UnivariateSpline(x, rrlist, k=3)


(1124, 'S30', 2.0, array([-2799.93818758, -2746.8038788 , -2691.98711277, ...,
          33.86421058,    28.44091637,    23.60779911]), {'bpm': 128.58888751129138, 'ibi': 466.60330578512395, 'sdnn': 57.0239154322162, 'sdsd': 57.344200816029876, 'rmssd': 75.51673839472794, 'pnn20': 0.6239316239316239, 'pnn50': 0.3247863247863248, 'hr_mad': 32.0, 'sd1': 53.364450981283966, 'sd2': 57.91298598331542, 's': 9709.07545077624, 'sd1/sd2': 0.9214591524715738, 'breathingrate': 0.23333333333333334})
Process end: S30
1
(1125, 'S31', 2.0, array([-3594.66056444, -3561.03846372, -3528.19266962, ...,
         -69.11987598,   -58.18724799,   -48.40776988]), {'bpm': 161.34996134323842, 'ibi': 371.8625, 'sdnn': 43.61084834934996, 'sdsd': 31.33988327501093, 'rmssd': 38.33787230442228, 'pnn20': 0.4088050314465409, 'pnn50': 0.06289308176100629, 'hr_mad': 26.5, 'sd1': 27.09272417998496, 'sd2': 51.99479097788077, 's': 4425.500406689625, 'sd1/sd2': 0.5210661235566951, 'breathingrate': 0.23333333333333334})
Proc

In [29]:
df_metadata = pd.DataFrame(result_metadata)
#df_metadata.dropna(inplace=True)
df_metadata

,id,age_group,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,1097,1.0,121.055354,495.641026,76.222679,75.016076,103.935335,0.784615,0.476923,46.0,73.220156,62.235014,14315.789752,1.176511,0.233333
1,1098,1.0,175.006286,342.844828,25.831106,9.963574,16.715920,0.219653,0.005780,17.0,11.818693,34.589335,1284.285398,0.341686,0.166667
2,1099,1.0,172.026424,348.783626,35.246251,25.931360,32.056935,0.294118,0.070588,18.0,22.655031,41.958326,2986.294905,0.539941,0.166667
3,1100,1.0,162.954230,368.201550,101.253338,116.016677,144.460289,0.650000,0.391667,28.0,102.108201,100.084194,32105.245183,1.020223,0.300000
4,1101,1.0,143.615676,417.781690,44.288764,37.912017,50.716847,0.617021,0.177305,23.5,35.844628,48.175058,5424.955715,0.744050,0.166667
5,1102,1.0,179.501505,334.259036,57.877616,71.288230,81.420426,0.484472,0.142857,20.0,57.568150,57.167139,10339.001763,1.007015,0.200000
6,1103,1.0,137.473164,436.448819,65.597161,45.972722,64.450649,0.666667,0.349593,42.0,45.533988,79.950291,11436.828492,0.569529,0.200000
7,1104,1.0,138.009771,434.751825,46.925905,17.879654,29.452479,0.433824,0.095588,35.0,20.825762,63.034956,4124.128779,0.330384,0.233333
8,1105,1.0,197.796243,303.342466,112.567206,118.182951,144.736798,0.459259,0.281481,22.5,101.919777,110.139208,35265.519758,0.925372,0.200000
9,1106,1.0,150.489850,398.697987,51.329271,48.846372,55.520267,0.378378,0.054054,26.0,39.241445,58.274289,7184.091358,0.673392,0.233333


In [30]:
df_metadata.to_csv(r"D:\Proga\AML\PPG_dataset\dataset.csv",mode="a",index=False,header=False)

In [31]:
df_signals = pd.DataFrame(result_signals)
df_signals

,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,...,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147
0,17153.260618,-579.054173,-269.257426,-5082.186014,981.010295,-226.557905,-1726.618990,2347.016257,-154.234785,82.585547,...,-714.065454,-6926.273273,-6081.869098,4555.149791,-653.357906,-3433.095430,-2788.158679,-12921.402603,10380.961949,-411.012663
1,17178.794805,-612.778486,-296.784710,-5161.369724,978.947046,-240.786058,-1612.724383,2341.097688,-187.192170,113.801293,...,-744.326507,-6913.405139,-6115.921653,4538.173179,-576.159997,-3423.866054,-2919.208548,-12938.010366,10320.039428,-378.944390
2,17203.249606,-646.784384,-324.482273,-5239.663884,976.812645,-255.134289,-1497.641520,2334.969410,-220.733832,145.865826,...,-775.005045,-6898.820075,-6149.242965,4520.391287,-497.614337,-3413.446967,-3051.662728,-12951.904233,10257.789388,-346.740330
3,17226.604426,-681.061679,-352.339033,-5317.016839,974.607118,-269.600926,-1381.403133,2328.632261,-254.859401,178.787174,...,-806.098727,-6882.487248,-6181.800481,4501.796039,-417.727919,-3401.812658,-3185.495745,-12963.016706,10194.214613,-314.412938
4,17248.838791,-715.599889,-380.343650,-5393.376524,972.330572,-284.184147,-1264.043391,2322.087173,-289.568169,212.573044,...,-837.604994,-6864.376161,-6213.561294,4482.379651,-336.508666,-3388.937551,-3320.681171,-12971.280340,10129.318044,-281.974875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,-3.643600,9.158170,-5.269445,-109.429490,2.718722,13.796410,24.718709,-43.277047,39.828801,19.882467,...,-90.950052,22.098175,-164.140685,-0.556653,41.794649,22.960160,49.673772,44.417704,39.767939,-23.964966
59996,-3.104725,7.855154,-4.766248,-93.404439,2.322995,11.826395,21.340038,-36.984593,33.989324,17.008938,...,-78.365699,18.909668,-140.704203,-0.199605,35.719390,19.649465,42.554946,38.019047,34.121465,-20.543583
59997,-2.621589,6.675447,-4.261285,-78.986680,1.966392,10.043959,18.251482,-31.312951,28.737250,14.416350,...,-66.894117,16.032161,-119.490448,0.063355,30.245488,16.660333,36.119906,32.241753,29.004666,-17.447874
59998,-2.191162,5.614337,-3.762124,-66.101042,1.647163,8.441851,15.447210,-26.234917,24.044897,12.092577,...,-56.507247,13.452302,-100.417678,0.246592,25.345743,13.979343,30.341081,27.059667,24.398832,-14.665298


In [ ]:
df_signals.to_csv(r"D:\Proga\AML\PPG_dataset\signals_lucia.csv",index = False)

AttributeError: module 'pandas' has no attribute 'to_csv'